In [1]:
!pip install opencv-python
!pip install pytest-shutil

### Import libraries

In [4]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import model_from_json
import random
from shutil import copyfile

In [4]:
tf.__version__

'1.14.0'

### Number of images in our Dataset

In [3]:
print("Number of images with mask used: ", len(os.listdir('../dataset/with_mask')))
print("Number of images without mask used: ", len(os.listdir('../dataset/without_mask')))

Number of images with mask used:  794
Number of images without mask used:  707


### Spliting the data into training and test set

In [6]:
def split_data(source, train, test, test_size):
    data = os.listdir(source)
    train_size = int(len(data)*(1 - test_size))
    shuffled_data = random.sample(data, len(data))
    training_data = shuffled_data[:train_size]
    testing_data = shuffled_data[train_size:]
    
    # make new folders train and test
    for img in training_data:
        temp_image = source+img
        train_image = train+img
        copyfile(temp_image, train_image)
    
    for img in testing_data:
        temp_image = source+img
        test_image = test+img
        copyfile(temp_image, test_image)
    

In [8]:
MASK_SOURCE = '../dataset/with_mask/'
TRAIN_MASK_SOURCE = '../dataset/train/yes/'
TEST_MASK_SOURCE = '../dataset/test/yes/'

NO_MASK_SOURCE = '../dataset/without_mask/'
TEST_NO_MASK_SOURCE = '../dataset/test/no/'
TRAIN_NO_MASK_SOURCE = '../dataset/train/no/'

split_data(MASK_SOURCE, TRAIN_MASK_SOURCE, TEST_MASK_SOURCE, 0.2)
split_data(NO_MASK_SOURCE, TRAIN_NO_MASK_SOURCE, TEST_NO_MASK_SOURCE, 0.2)

In [9]:
print('Number of training with mask: ', len(os.listdir(TRAIN_MASK_SOURCE)))
print('Number of training without mask: ', len(os.listdir(TRAIN_NO_MASK_SOURCE)))
print('Number of testing with mask: ', len(os.listdir(TEST_MASK_SOURCE)))
print('Number of testing without mask: ', len(os.listdir(TEST_NO_MASK_SOURCE)))

Number of training with mask:  635
Number of training without mask:  565
Number of testing with mask:  159
Number of testing without mask:  142


### Preprocessing data

In [10]:
TRAIN_DIR = '../dataset/train/'
TEST_DIR = '../dataset/test/'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (150,150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (150,150),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1200 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


### Building CNN

In [11]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Training the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
# Used to get best weights with monitored with validation loss
checkpoint = ModelCheckpoint('.ipynb_checkpoints/models_checkpoints/model-{epoch:03d}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

In [43]:
# To train the modek on your computer and dataset
model = cnn.fit(x = training_set, validation_data = test_set, epochs = 30, callbacks=[checkpoint])

Epoch 1/30
38/38 [==============================] - 62s 2s/step - loss: 0.1399 - acc: 0.9400 - val_loss: 0.0818 - val_acc: 0.9734
Epoch 2/30
38/38 [==============================] - 51s 1s/step - loss: 0.1465 - acc: 0.9467 - val_loss: 0.0924 - val_acc: 0.9734
Epoch 3/30
38/38 [==============================] - 52s 1s/step - loss: 0.1416 - acc: 0.9408 - val_loss: 0.1029 - val_acc: 0.9701
Epoch 4/30
38/38 [==============================] - 52s 1s/step - loss: 0.1486 - acc: 0.9458 - val_loss: 0.1067 - val_acc: 0.9601
Epoch 5/30
38/38 [==============================] - 52s 1s/step - loss: 0.1280 - acc: 0.9483 - val_loss: 0.0911 - val_acc: 0.9668
Epoch 6/30
38/38 [==============================] - 56s 1s/step - loss: 0.1606 - acc: 0.9392 - val_loss: 0.0904 - val_acc: 0.9734
Epoch 7/30
38/38 [==============================] - 54s 1s/step - loss: 0.1358 - acc: 0.9442 - val_loss: 0.0787 - val_acc: 0.9668
Epoch 8/30
38/38 [==============================] - 53s 1s/step - loss: 0.1221 - acc: 0.95

In [14]:
training_set.class_indices

{'no': 0, 'yes': 1}

In [15]:
# To save the model in your disk
'''

# serialize model to JSON
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model.h5")
print("Saved model to disk")
'''

'\n\n# serialize model to JSON\nmodel_json = cnn.to_json()\nwith open("model.json", "w") as json_file:\n    json_file.write(model_json)\n# serialize weights to HDF5\ncnn.save_weights("model.h5")\nprint("Saved model to disk")\n'

In [69]:
# Uncomment this to directly load the trained model if you don't wish to train it on your computer

# load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)


# load weights into new model
cnn.load_weights("model.h5")
print("Loaded model from disk")


Loaded model from disk


### Deploy the model on webcam input

In [76]:
labels = ['No Mask :/', 'Mask On!', 'By Shashank Singhaina']
grid_color = [(0, 0, 255), (255, 0, 0)]

print("Press esc or q to quit")

# If you have multiple webcameras, feel free to change the source
# 0 is for default camera...
# 1 is for secondary camera...
webcam = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier('C:/Users/Shashank/anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
try: 
    while webcam.isOpened():
        _, frame = webcam.read()
        frame = cv2.flip(frame, 1, 1)
        faces = classifier.detectMultiScale(frame, 1.1, 4)

        for (x,y,w,h) in faces:
            face = frame[y-40:y+h+20, x-20:x+w+20]
            try:
                resized_data = (cv2.resize(face, (150, 150)))/255.0
            except Exception as e:
                resized_data = (cv2.resize(frame, (150, 150)))/255.0
            final_data = np.expand_dims(resized_data, axis = 0)
            prediction = cnn.predict(final_data)
            answer = prediction[0][0] #Binary answer 0 = no, 1 = yes
            # print(answer)
            answer = int(answer < 0.2) #Can be changed according to your picture
            
            
            # display the answer
            cv2.rectangle(frame, (x,y), (x+w, y+h), grid_color[answer], 3)
            cv2.rectangle(frame, (x-2, y-45), (x+w,y), grid_color[answer], -1)
            cv2.putText(frame, labels[answer], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
            cv2.putText(frame, labels[2], (250 ,470), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)

        cv2.imshow("MASK DETECTOR - by SHASHANK SINGHANIA", frame)
        key = cv2.waitKey(1)
        if key == 27 or (key & 0xFF == ord('q')):
            break
except:
    raise
    
finally:        
    webcam.release()
    cv2.destroyAllWindows()
        

Press esc or q to quit


### Test Without Webcam

In [74]:
from keras.preprocessing import image
test_image = image.load_img('test_data/img4.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result[0][0])
res = "Mask on" if (result[0][0]>0.4) else "Mask off" 
print(res)



1.0
Mask on
